In [18]:
from sentinelhub import (
    SHConfig,
    BBox,
    CRS,
    bbox_to_dimensions,
    DataCollection,
    MimeType,
    MosaickingOrder,
    SentinelHubRequest,
)

def get_images(bbox):
    
    # put here your sentinel hub credential
    config = SHConfig()
    config.sh_client_id = "0bd42ccb-b998-46c5-9f7c-2161d40e600d"
    config.sh_client_secret = "4ssKzKjq9KOwgBlCGjNSQsR2Dr0naY1K"

    # customize this functions to specify the band you want to download
    evalscript = """
        //VERSION=3
        function setup() {
            return {
                input: [{
                    bands: ["B08", "B11"],
                    units: "DN"
                }],
                output: {
                    bands: 2,
                    sampleType: "INT16"
                }
            };
        }

        function evaluatePixel(sample) {
            return [
                sample.B08,
                sample.B11
            ];
        }
    """

    # put here the coordinate of the image you want to download
    # coordinates = (-60.445274, -3.403784, -60.3, -2.490358)

    # set the coordinate reference system
    # crs = CRS.WGS84

    # set the desired resolution
    resolution = 7

    # set the time interval
    time_interval = ("2023-08-01", "2023-08-31")

    # set the source of the image
    data_collection = DataCollection.SENTINEL2_L2A

    # set the constant class for type of mosaicking order
    mosaicking_order = MosaickingOrder.LEAST_CC

    # update this value if you want to save the image
    save_data = True

    # set the folder where you want to dowload the image
    data_folder = './data/sentinel_images'

    # set the format of the image
    response_format = MimeType.TIFF

    # bbox = BBox(bbox = coordinates, crs = crs)
    size = bbox_to_dimensions(bbox, resolution = resolution)
    request = SentinelHubRequest(
        config = config,
        bbox = bbox,
        size = size,
        evalscript = evalscript,
        data_folder = data_folder,
        input_data = [
            SentinelHubRequest.input_data(
                time_interval = time_interval,
                data_collection = data_collection,
                mosaicking_order = mosaicking_order
            )
        ],
        responses = [
            SentinelHubRequest.output_response(
                identifier = "default",
                response_format = response_format
            )
        ]
    )

    return request.get_data(save_data = save_data)

In [25]:
import folium
import geojson
from sentinelhub import (
    BBoxSplitter,
    CRS
)
import math 
import geopy.distance
from shapely.geometry import Polygon

map = folium.Map((-10.961745, -42.387077), zoom_start=7)

x_min = 180
x_max = -180
y_min = 90
y_max = -90

with open('../veg-monitor/datasets_analysis/data/towers/balbina_powerline_dataframe.geojson') as f:
    dataframe = geojson.load(f)
for powerline in dataframe['features']:
    if powerline['geometry'] is not None:
        for i in range(len(powerline['geometry']['coordinates']) - 1):
            if powerline['geometry']['coordinates'][i][0] < x_min:
                x_min = powerline['geometry']['coordinates'][i][0]
            if powerline['geometry']['coordinates'][i][0] > x_max:
                x_max = powerline['geometry']['coordinates'][i][0]
            if powerline['geometry']['coordinates'][i][1] < y_min:
                y_min = powerline['geometry']['coordinates'][i][1]
            if powerline['geometry']['coordinates'][i][1] > y_max:
                y_max = powerline['geometry']['coordinates'][i][1]

            folium.PolyLine(
                locations=[
                    (powerline['geometry']['coordinates'][i][1],
                    powerline['geometry']['coordinates'][i][0]),
                    (powerline['geometry']['coordinates'][i + 1][1],
                    powerline['geometry']['coordinates'][i + 1][0]),
                ]
            ).add_to(map)

padding = 0.2

x_min -= padding
x_max += padding
y_min -= padding
y_max += padding

coordinates = (x_min, y_min, x_max, y_max)

polygon = Polygon([
    [coordinates[0], coordinates[1]],
    [coordinates[2], coordinates[1]],
    [coordinates[2], coordinates[3]],
    [coordinates[0], coordinates[3]],
    [coordinates[0], coordinates[1]]
])

bounds = polygon.bounds

max_pixels = 2500
resolution = 7

distance_x = geopy.distance.geodesic((bounds[1], bounds[0]), (bounds[1], bounds[2])).m
needed_pixels_x = distance_x / resolution
n_bbox_x = math.ceil(needed_pixels_x / max_pixels) + 1

distance_y = geopy.distance.geodesic((bounds[1], bounds[0]), (bounds[3], bounds[0])).m
needed_pixels_y = distance_y / resolution
n_bbox_y = math.ceil(needed_pixels_y / max_pixels) + 1

bbox_splitter = BBoxSplitter([polygon], CRS('4326'), (n_bbox_x, n_bbox_y))

bboxs = []
for bbox in bbox_splitter.get_bbox_list():
    trovato = False
    for powerline in dataframe['features']:
        if powerline['geometry'] is not None:
            for i in range(len(powerline['geometry']['coordinates'])):
                if powerline['geometry']['coordinates'][i][0] >= bbox.min_x and \
                        powerline['geometry']['coordinates'][i][0] <= bbox.max_x and \
                        powerline['geometry']['coordinates'][i][1] >= bbox.min_y and \
                        powerline['geometry']['coordinates'][i][1] <= bbox.max_y:
                    bboxs.append(bbox)

for i in range(len(bboxs)):
    print('\r', int((i+1) / len(bboxs) * 100), '%', sep='', end='')
    get_images(bboxs[i])
    folium.Rectangle([(bboxs[i].max_y, bboxs[i].min_x), (bboxs[i].min_y, bboxs[i].max_x)]).add_to(map)

map

100.0%99999999999%%

In [26]:
import folium
import os
import rasterio

map = folium.Map()
for path in os.listdir('data/sentinel_images/'):
    with rasterio.open('data/sentinel_images/' + path + '/response.tiff') as image:
        left, bottom, right, top = image.bounds
        folium.Rectangle([(top, left), (bottom, right)]).add_to(map)
map